In [ ]:
#IMPORTS OF PACKAGES
import mne
from os.path import join
import matplotlib.pyplot as plt
import pathlib

plt.ion() #toggle interactive plotting
# import numpy as np

In [ ]:
# PATHS
path = pathlib.Path.cwd()
meg_path = path.parents[3] / "834761" / "0114" / "20230927_000000" / "MEG" / "001.self_block1" / "files"

bem_path = path.parents[3] / "835482" / "0114" / "bem"

subjects_dir = path.parents[3] / "835482" 

raw_name = 'self_block1.fif'
fwd_name = 'self_block1-oct-6-src-5120-fwd.fif'